<a href="https://colab.research.google.com/github/DhruboDevPramanik/CharacterByFace/blob/main/Copy_of_CharacterPredictionByFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
gpiosenka_good_guysbad_guys_image_data_set_path = kagglehub.dataset_download('gpiosenka/good-guysbad-guys-image-data-set')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout


In [ ]:
# generators, since we have a very large data set so we convert it into small batches(32)
train_ds = keras.utils.image_dataset_from_directory(        # library we have to import
    directory = '/kaggle/input/good-guysbad-guys-image-data-set/train',  # path of train data
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/good-guysbad-guys-image-data-set/valid',     # yha per test data ka path hai
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

test_ds = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/good-guysbad-guys-image-data-set/test',     # yha per test data ka path hai
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

In [ ]:
# Normalize  # we have to  make aur pixel in same range(0,1)
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)
test_ds = test_ds.map(process)

In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Use this if labels are integers
    metrics=['accuracy']
)


## call back
from keras.callbacks import ModelCheckpoint , EarlyStopping

# Callbacks
mc = ModelCheckpoint(
    monitor='val_accuracy',           # Metric to monitor
    filepath='bestmodel.keras',       # Save best model to .keras format
    verbose=1,                        # Verbosity mode
    save_best_only=True,               # Save only the best model
    mode='auto'                       # Mode for monitoring (auto, min, max)
)
es = EarlyStopping(
    monitor='val_accuracy',           # Metric to monitor
    min_delta=0.01,                   # Minimum change to qualify as an improvement
    patience=3,                       # Number of epochs with no improvement after which training will be stopped
    verbose=1,                        # Verbosity mode
    mode='auto'                       # Mode for monitoring (auto, min, max)
)


cb = [mc , es]

history = model.fit(train_ds,
                              steps_per_epoch = 30 ,
                              epochs = 30,
                              validation_data = validation_ds,
                              #validation_step = 16,
                              callbacks = cb)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()



plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.mobilenet import preprocess_input

# Define the image path
path = "/kaggle/input/good-guysbad-guys-image-data-set/train/unsavory/0016.jpg"

# Load and preprocess the image
img = load_img(path, target_size=(256, 256))
input_arr = img_to_array(img)
input_arr = preprocess_input(input_arr)  # Use the preprocessing function for MobileNet

# Visualize the image
plt.imshow((input_arr + 1) / 2)  # Display the image with correct scaling; MobileNet preprocess_input can result in values in [-1, 1]
plt.axis('off')  # Hide axis
plt.show()

# Check the shape of the array
print("Shape of input array:", input_arr.shape)

# Add a batch dimension
input_arr = np.expand_dims(input_arr, axis=0)

# Make prediction
predictions = model.predict(input_arr)

# Get the predicted class
predicted_class_index = np.argmax(predictions[0])  # Get the index of the highest probability
predicted_class_probability = np.max(predictions[0])  # Get the probability of the predicted class

print("Predicted class index:", predicted_class_index)
print("Predicted class probability:", predicted_class_probability)

# Map predicted class index to class name (assuming you have a list of class names)
class_names = ['savory', 'unsaavory']  # Replace with your actual class names
predicted_class_name = class_names[predicted_class_index]

print("Predicted class name:", predicted_class_name)